# State Management with StateMachine
This notebook demonstrates how to work with state management using a StateMachine implementation. We'll explore how to create, manage, and control workflow states in a structured way.

## What we'll learn:
- Basic state machine concepts and implementation
- Creating and connecting workflow steps
- Managing state transitions and data flow
- Working with routing and loops in state machines
- Understanding state snapshots and execution flow

### Setup

In [1]:
from typing import TypedDict
from lib.state_machine import (
    StateMachine,
    Step,
    EntryPoint,
    Termination,
)

## Basic State Machine Concepts
Let's start with a simple example that demonstrates the core concepts of our state machine:
1. Defining state schema
2. Creating steps
3. Connecting steps
4. Running the workflow

**Creating the Schema and the State Machine**

In [2]:
class Schema(TypedDict):
    """Schema defining the structure of our state.
    
    Attributes:
        input: The input value to process
        output: The processed output value
    """
    input: int
    output: int

In [3]:
# Create our state machine instance
workflow = StateMachine(Schema)

**Defining the logic for Steps**

In [4]:
def step_input(state: Schema) -> Schema:
    """First step: Increment the input value.
    
    Args:
        state: Current state containing input value
        
    Returns:
        Updated state with incremented value in output
    """
    return {"output": state["input"] + 1, "random": 10}



In [5]:
def step_double(state: Schema) -> Schema:
    """Second step: Double the previous output.
    
    Args:
        state: Current state containing output from previous step
        
    Returns:
        Updated state with doubled output value
    """
    return {"output": state["output"] * 2}


**Creating and Connecting Steps**

In [6]:
entry = EntryPoint()
s1 = Step("input", step_input)
s2 = Step("double", step_double)
termination = Termination()

In [7]:
workflow.add_steps([entry, s1, s2, termination])

In [8]:
workflow.connect(entry, s1)
workflow.connect(s1, s2)
workflow.connect(s2, termination)

In [9]:
workflow.transitions

{'__entry__': [Transition('__entry__' -> ['input'])],
 'input': [Transition('input' -> ['double'])],
 'double': [Transition('double' -> ['__termination__'])]}

**Running the Workflow**

In [10]:
initial_state = {"input": 4}
run_object = workflow.run(initial_state)
run_object

[StateMachine] Starting: __entry__
[StateMachine] Executing step: input
[StateMachine] Executing step: double
[StateMachine] Terminating: __termination__


Run('9a73ad0f-72c9-44b0-af07-76ff0914b6a4')

In [11]:
run_object.snapshots

[Snapshot('7400fa01-e35a-4912-9cde-bfce1632ff36') @ [2025-11-03 21:33:33.439306]: __entry__.State({'input': 4}),
 Snapshot('eb2fa760-6c70-4ee1-90a5-6f0a6767e1df') @ [2025-11-03 21:33:33.439361]: input.State({'input': 4, 'output': 5}),
 Snapshot('09533f44-c0c8-4794-bb3e-b618ed0e77df') @ [2025-11-03 21:33:33.439399]: double.State({'input': 4, 'output': 10})]

## Advanced State Management: Routing and Loops
Now we'll explore more complex state management patterns including:
- Conditional routing between steps
- Creating loops in the workflow
- Managing state through multiple iterations

In [12]:
class CounterSchema(TypedDict):
    """Schema for a counter-based workflow.
    
    Attributes:
        count: Current counter value
        max_value: Maximum value before termination
    """
    count: int
    max_value: int

In [13]:
workflow = StateMachine(CounterSchema)

In [14]:
def increment_counter(state: CounterSchema) -> CounterSchema:
    """Increment the counter value.
    
    Args:
        state: Current state with counter value
        
    Returns:
        Updated state with incremented counter
    """
    return {"count": state["count"] + 1}

In [15]:
# Create steps
entry = EntryPoint()
increment = Step("increment", increment_counter)
termination = Termination()

In [16]:
workflow.add_steps([entry, increment, termination])

In [17]:
# Router logic
def check_counter(state: CounterSchema) -> Step:
    """Determine next step based on counter value.
    
    Args:
        state: Current state with counter and max value
        
    Returns:
        Next step to execute (increment or terminate)
    """
    if state["count"] >= state["max_value"]:
        return termination
    return increment

In [18]:
# Connect steps with a loop in increment
workflow.connect(entry, increment)
workflow.connect(increment, [increment, termination], check_counter)

In [19]:
workflow.transitions

{'__entry__': [Transition('__entry__' -> ['increment'])],
 'increment': [Transition('increment' -> ['increment', '__termination__'])]}

In [20]:
initial_state = {"count": 0, "max_value": 3}
run_object = workflow.run(initial_state)
run_object

[StateMachine] Starting: __entry__
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Terminating: __termination__


Run('dc89a600-8a61-49e3-bad7-3202740a71a0')

In [21]:
run_object.snapshots

[Snapshot('388d6ed2-654b-4c27-a693-6b88afca34ac') @ [2025-11-03 21:33:41.983622]: __entry__.State({'count': 0, 'max_value': 3}),
 Snapshot('54d4f9da-2bca-4e2c-a1cd-0705147848f7') @ [2025-11-03 21:33:41.983669]: increment.State({'count': 1, 'max_value': 3}),
 Snapshot('e40a7702-9fac-4305-96be-3f7181b2f35d') @ [2025-11-03 21:33:41.983708]: increment.State({'count': 2, 'max_value': 3}),
 Snapshot('babf6b12-8b66-4a02-a1c4-9305e07d29f3') @ [2025-11-03 21:33:41.983744]: increment.State({'count': 3, 'max_value': 3})]

In [22]:
run_object.get_final_state()

{'count': 3, 'max_value': 3}